### 2019/07/19 meeting Notes
-
1. raw data拿到就先分<span style="color:red">**大中小公司**</span>，再去做基本統計量
2. 第三部分的實作需要<span style="color:red">**價量資料**</span>
3. 資料的部分，最早應該要從2012/1/6開始抓

<img src="https://i.imgur.com/YZ8f21o.jpg" height="400" width="400">
<img src="https://i.imgur.com/VRp7NCJ.jpg" height="400" width="400">

### Document

<img src="https://i.imgur.com/1giFKNu.png">

### 第一部分：
- 基本統計量說明: Raw Data在當沖前/當沖後的各種基本統計量陳述（平均數、中位數、min、max、標準差），對Raw Data有個基本的概念

### 第二部分：
- 當沖前/當沖後的各種基本統計量的檢定（ex:現股當沖比重、日報酬率、週報酬率、月報酬率）
- 做平均數檢定、中位數檢定，看看當沖前/當沖後這些基本統計量有沒有顯著變化

### 第三部分：
- 政策面的研究。政府開放當沖是為了縮小spread、提高成交量、提高成交量週轉率。
- 研究data做的回歸是不是符合政府所宣稱的?當沖真的有穩定市場嗎？對資本市場有貢獻嗎？

### 第四部分：
- 市場面研究。有沒有其他在市場上面的因素會影響我們的回歸式？要控制這些變數，放入回歸式（很多x)

### $ \Delta $ $上市公司日報酬率標準差_i$ = $a_0$ + $a_1$ * (上市公司現股當沖比重平均)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
import datetime as datetime
import json
import os
from datetime import timedelta
warnings.filterwarnings("ignore")
%matplotlib inline

from sklearn.linear_model import LinearRegression
from scipy import stats

### tw_day_trade class 

In [11]:
class tw_day_trade():
    def __init__(self):
        self.pos_1 = ""
        self.pos_2 = ""
        self.pos_3 = ""
        self.df_before = "" 
        self.df_after = ""  
        self.basic_stock_stats_info_dict = {}
        self.x = "" 
        self.y_daily = "" 
        self.y_weekly = "" 
        self.y_monthly = "" 

        """
        data basic info
        """
        self.day_trade_data = pd.read_csv("20130613_20190620_data.csv")
        self.company_num = len(self.day_trade_data)/4
        print("data 台灣上市公司總數: {} 間".format(self.company_num))
        print("data shape: {}".format(self.day_trade_data.shape))

        """
        當沖相關重要日期們
        """
        self.open_single_day_trade_date = '2014/1/6' # 開放單向當沖
        self.open_double_day_trade_date = '2014/6/30' # 開放雙向當沖
        self.tax_down_day_trade_date = '2017/4/28' # 調降當沖稅率
        
    
    def day_trade_split(self, date_1, date_2, date_3):
        """
        根據date_1, date_2, date_3來切割dataframe
        """
        tmp_arr = np.array(list(self.day_trade_data))
        self.pos_1 = np.where(tmp_arr==date_1)[0][0]
        self.pos_2 = np.where(tmp_arr==date_2)[0][0]
        self.pos_3 = np.where(tmp_arr==date_3)[0][0]
        print("date_1 pos: {}\ndate_2 pos: {}\ndate_3 pos: {}".format(self.pos_1,self.pos_2,self.pos_3))
        
        self.df_before = self.day_trade_data.iloc[:,self.pos_1:self.pos_2]
        self.df_after = self.day_trade_data.iloc[:,self.pos_2:(self.pos_3+1)]
        print("df_before shape: {}".format(self.df_before.shape))
        print("df after shape: {}".format(self.df_after.shape))
        
        
    def day_trade_stats(self):
        """
        第一部分：計算當沖前/當沖後的基本統計量，對Raw Data有個基本概念
        """
        
        # 平均數 (axis=1橫條往右apply func., axis=0是直條往下apply func.)
        self.df_before['mean_before'] = self.df_before.mean(axis=1)
        self.df_after['mean_after'] = self.df_after.mean(axis=1)
        
        test = self.df_before['mean_before']
        print(test)
        print(len(test))
        
        mean_before_daily = self.df_before['mean_before'][0::4].mean()
        mean_after_daily = self.df_after['mean_after'][0::4].mean()
        mean_before_weekly = self.df_before['mean_before'][1::4].mean()
        mean_after_weekly = self.df_after['mean_after'][1::4].mean()
        mean_before_monthly = self.df_before['mean_before'][2::4].mean()
        mean_after_monthly = self.df_after['mean_after'][2::4].mean()
        
        # 標準差 
        self.df_before['std_before'] = self.df_before.std(axis=1)
        self.df_after['std_after'] = self.df_after.std(axis=1)
        
        std_before_daily = self.df_before['std_before'][0::4].mean()
        std_after_daily = self.df_after['std_after'][0::4].mean()
        std_before_weekly = self.df_before['std_before'][1::4].mean()
        std_after_weekly = self.df_after['std_after'][1::4].mean()
        std_before_monthly = self.df_before['std_before'][2::4].mean()
        std_after_monthly = self.df_after['std_after'][2::4].mean()
        
        # 中位數
        self.df_before['median_before'] = self.df_before.median(axis=1)
        self.df_after['median_after'] = self.df_after.median(axis=1)
        
        median_before_daily = self.df_before['median_before'][0::4].mean()
        median_after_daily = self.df_after['median_after'][0::4].mean()
        median_before_weekly = self.df_before['median_before'][1::4].mean()
        median_after_weekly = self.df_after['median_after'][1::4].mean()
        median_before_monthly = self.df_before['median_before'][2::4].mean()
        median_after_monthly = self.df_after['median_after'][2::4].mean()
        
        # max
        max_before_daily = self.df_before['mean_before'][0::4].max()
        max_after_daily = self.df_after['mean_after'][0::4].max()
        max_before_weekly = self.df_before['mean_before'][1::4].max()
        max_after_weekly = self.df_after['mean_after'][1::4].max()
        max_before_monthly = self.df_before['mean_before'][2::4].max()
        max_after_monthly = self.df_after['mean_after'][2::4].max()
        
        # min
        min_before_daily = self.df_before['mean_before'][0::4].min()
        min_after_daily = self.df_after['mean_after'][0::4].min()
        min_before_weekly = self.df_before['mean_before'][1::4].min()
        min_after_weekly = self.df_after['mean_after'][1::4].min()
        min_before_monthly = self.df_before['mean_before'][2::4].min()
        min_after_monthly = self.df_after['mean_after'][2::4].min()
        
        
        # 存放基本統計量的dictionary
        self.basic_stock_stats_info_dict = {
            'mean':{'mean_before_daily':mean_before_daily,'mean_after_daily':mean_after_daily,
                   'mean_before_weekly':mean_before_weekly,'mean_after_weekly':mean_before_weekly,
                   'mean_before_monthly':mean_before_monthly,'mean_before_monthly':mean_before_monthly},
            
            'std':{'std_before_daily':std_before_daily, 'std_after_daily':std_after_daily,
                  'std_before_weekly':std_before_weekly, 'std_after_weekly':std_after_weekly,
                  'std_before_monthly':std_before_monthly, 'std_after_monthly':std_after_monthly},
            
            'median':{'median_before_daily':median_before_daily, 'median_after_daily':median_after_daily,
                  'median_before_weekly':median_before_weekly, 'median_after_weekly':median_after_weekly,
                  'median_before_monthly':median_before_monthly, 'median_after_monthly':median_after_monthly},
            
            'max':{'max_before_daily':max_before_daily, 'max_after_daily':max_after_daily,
                  'max_before_weekly':max_before_weekly, 'max_after_weekly':max_after_weekly,
                  'max_before_monthly':max_before_monthly, 'max_after_monthly':max_after_monthly},
            
            'min':{'min_before_daily':min_before_daily, 'min_after_daily':min_after_daily,
                  'min_before_weekly':min_before_weekly, 'min_after_weekly':min_after_weekly,
                  'min_before_monthly':min_before_monthly, 'min_after_monthly':min_after_monthly},

        }
        
        print (json.dumps(self.basic_stock_stats_info_dict, indent=2))
        
    
    def day_trade_calculate(self):
        """
        準備OLS的x和y
        """
        self.x = self.df_after['mean_after'][3::4]-self.df_before['mean_before'][3::4]
        self.y_daily = self.df_after['std_after'][0::4]-self.df_before['std_before'][0::4]
        self.y_weekly = self.df_after['std_after'][1::4]-self.df_before['std_before'][1::4]
        self.y_monthly = self.df_after['std_after'][2::4]-self.df_before['std_before'][2::4]
    
    def day_trade_OLS(self, date_freq):
        """
        根據傳進的date_freq，計算OLS(日/週/月)
        date_freq's value: 'daily','weekly','monthly'
        """
        if date_freq=='daily':
            self.x = sm.add_constant(self.x)
            self.y_daily = list(self.y_daily)
            model_daily = sm.OLS(self.y_daily, self.x).fit()
            print(model_daily.summary())

        if date_freq=='weekly':
            self.x = sm.add_constant(self.x)
            self.y_weekly = list(self.y_weekly)
            model_weekly = sm.OLS(self.y_weekly, self.x).fit()
            print(model_weekly.summary())

        if date_freq=='monthly':
            self.x = sm.add_constant(self.x)
            self.y_monthly = list(self.y_monthly)
            model_monthly = sm.OLS(self.y_monthly, self.x).fit()
            print(model_monthly.summary())


### 2014/1/6開放單向當沖，前後各抓半年
#### **<span style="color:green">pos_1 = 2013/6/13, pos_2 = 2014/1/6, pos_3 = 2014/6/30</span>**

In [12]:
data_1=tw_day_trade()
data_1.day_trade_split('2013/6/13','2014/1/6','2014/6/30')
data_1.day_trade_stats()
data_1.day_trade_calculate()
data_1.day_trade_OLS('daily')
data_1.day_trade_OLS('weekly')
data_1.day_trade_OLS('monthly')

data 台灣上市公司總數: 949.0 間
data shape: (3796, 1481)
date_1 pos: 2
date_2 pos: 145
date_3 pos: 260
df_before shape: (3796, 143)
df after shape: (3796, 116)
0       0.156810
1       0.861894
2       3.286883
3       0.000000
4       0.070137
5       0.368999
6       1.336310
7       0.000000
8       0.067669
9       0.298076
10      1.912097
11      0.000000
12      0.373745
13      1.778400
14      8.629051
15      0.000000
16      0.070683
17      0.293102
18      2.227292
19      0.000000
20      0.108421
21      0.556024
22      2.201441
23      0.000000
24      0.075921
25      0.381440
26      2.304257
27      0.000000
28      0.024615
29      0.172497
          ...   
3766    0.966558
3767    0.000000
3768    0.348920
3769    1.713836
3770    6.797857
3771    0.000000
3772   -0.074154
3773   -0.405136
3774   -1.698180
3775    0.000000
3776    0.285416
3777    1.469350
3778    5.962972
3779    0.000000
3780    0.049771
3781    0.333887
3782    1.307742
3783    0.000000
3784    0.123578

### 2014/6/30 開放雙向當沖，前後各抓半年
#### **<span style="color:green">pos_1 = 2013/12/30, pos_2 = 2014/6/30, pos_3 = 2014/12/30</span>**

In [13]:
data_2 = tw_day_trade()
data_2.day_trade_split('2013/12/30','2014/6/30','2014/12/30')
data_2.day_trade_stats()
data_2.day_trade_calculate()
data_2.day_trade_OLS('daily')
data_2.day_trade_OLS('weekly')
data_2.day_trade_OLS('monthly')

data 台灣上市公司總數: 949.0 間
data shape: (3796, 1481)
date_1 pos: 141
date_2 pos: 260
date_3 pos: 389
df_before shape: (3796, 119)
df after shape: (3796, 130)
0       -0.008354
1       -0.070458
2       -0.174300
3        1.694118
4        0.053984
5        0.266682
6        0.716839
7        0.884118
8        0.011463
9        0.022368
10       0.222653
11       0.000000
12      -0.005073
13       0.168734
14      -0.458891
15       0.000000
16       0.069553
17       0.398199
18       1.697206
19       0.000000
20       0.249248
21       0.966927
22       4.069623
23       0.000000
24      -0.085064
25      -0.341155
26      -1.388455
27       0.000000
28      -0.108411
29      -0.375744
          ...    
3766     0.403166
3767     0.000000
3768     0.326269
3769     1.615255
3770     8.779425
3771     0.000000
3772     0.183764
3773     1.118729
3774     3.591101
3775     0.000000
3776     0.695227
3777     3.772394
3778    14.824650
3779     0.000000
3780    -0.051521
3781    -0.260817
3

### 2014/6/30 開放雙向當沖，前後各抓一年
#### **<span style="color:green">pos_1 = 2013/6/28, pos_2 = 2014/6/30, pos_3 = 2015/6/30</span>**

In [14]:
data_3 = tw_day_trade()
data_3.day_trade_split('2013/6/28','2014/6/30','2015/6/30')
data_3.day_trade_stats()
data_3.day_trade_calculate()
data_3.day_trade_OLS('daily')
data_3.day_trade_OLS('weekly')
data_3.day_trade_OLS('monthly')

data 台灣上市公司總數: 949.0 間
data shape: (3796, 1481)
date_1 pos: 13
date_2 pos: 260
date_3 pos: 506
df_before shape: (3796, 247)
df after shape: (3796, 247)
0        0.124196
1        0.637545
2        2.192343
3        0.816194
4        0.082124
5        0.434077
6        1.366975
7        0.425951
8        0.052826
9        0.263215
10       1.079655
11       0.000000
12       0.250133
13       1.252101
14       5.019063
15       0.000000
16       0.095160
17       0.481299
18       2.056768
19       0.000000
20       0.184685
21       0.787819
22       3.173437
23       0.000000
24       0.016490
25       0.066929
26       0.329442
27       0.000000
28      -0.029340
29      -0.039102
          ...    
3766     1.159015
3767     0.000000
3768     0.345813
3769     1.723917
3770     8.006383
3771     0.000000
3772     0.035700
3773     0.314533
3774     0.592134
3775     0.000000
3776     0.518866
3777     2.592679
3778    10.387143
3779     0.000000
3780     0.036540
3781     0.243859
37

### 2017/4/28 調降當沖稅率，前後各抓一年
#### **<span style="color:green">pos_1 = 2016/4/28, pos_2 = 2017/4/28, pos_3 = 2018/4/27</span>**

In [15]:
data_4 = tw_day_trade()
data_4.day_trade_split('2016/4/28','2017/4/28','2018/4/27')
data_4.day_trade_stats()
data_4.day_trade_calculate()
data_4.day_trade_OLS('daily')
data_4.day_trade_OLS('weekly')
data_4.day_trade_OLS('monthly')

data 台灣上市公司總數: 949.0 間
data shape: (3796, 1481)
date_1 pos: 709
date_2 pos: 953
date_3 pos: 1199
df_before shape: (3796, 244)
df after shape: (3796, 247)
0       0.056067
1       0.266781
2       1.649248
3       3.828607
4       0.041682
5       0.195134
6       0.988857
7       2.641148
8       0.080683
9       0.295076
10      1.198403
11      1.825861
12      0.121865
13      0.588321
14      2.439920
15      1.105328
16      0.014234
17      0.024400
18      0.258802
19      0.851434
20      0.016281
21      0.072547
22      0.455898
23      0.982131
24      0.024072
25      0.095278
26      0.530577
27      4.807664
28     -0.035165
29     -0.172025
          ...   
3766    1.912000
3767    1.029508
3768    0.050307
3769    0.229749
3770    0.698109
3771    1.286557
3772    0.041012
3773    0.196164
3774    0.584765
3775    3.624836
3776   -0.027824
3777   -0.031884
3778   -0.295343
3779    4.594098
3780    0.008230
3781    0.090386
3782   -0.457835
3783    3.957213
3784   -0.053